In [33]:
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np

In [43]:
driver=webdriver.Chrome("C:/Users/Playdata/Desktop/chromedriver_win32/chromedriver.exe")
url="https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc"

C:\Users\Playdata\AppData\Local\Temp\ipykernel_19224\1645652240.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("C:/Users/Playdata/Desktop/chromedriver_win32/chromedriver.exe")


In [44]:
driver.get(url)

In [45]:
html=driver.page_source
len(html)

436482

In [46]:
soup=BeautifulSoup(html,'html.parser')
print(soup.prettify)

<bound method Tag.prettify of <html class="scriptsOn" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"><head><script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/31bJewCvY-L.js"></script>
<script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel

In [38]:
# movie_div=soup.find_all('div',class_='lister-item mode-advanced')
# for container in movie_div:
#     name=container.h3.a.text
#     print(name)

In [47]:
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')

In [48]:
one_movie_info = soup.select_one('div.lister-item.mode-advanced')
one_movie_info.h3.a['href'].split('/')[2]

AttributeError: 'NoneType' object has no attribute 'h3'

In [41]:
one_movie_info.find('span',class_='genre').text.strip()

'Drama'

In [49]:
import time
import numpy as np
names=[]
years=[]
runtimes=[]
imdb_ratings=[]
meta_scores=[]
genres=[]
movie_id=[]
pages=np.arange(1,251,50)
for page in pages:
    url=f'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start={page}&ref_=adv_nxt'
    time.sleep(3)
    driver.get(url)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    soup.prettify
    movies_info = soup.select('div.lister-item.mode-advanced')
    #이름
    for one_movie_info in movies_info:
        #제목
        names.append(one_movie_info.h3.a.get_text().strip())
        #연도
        years.append(one_movie_info.h3.find('span',class_='lister-item-year').text)
        #시간
        runtimes.append(one_movie_info.p.find('span',class_='runtime').text)
        #평점
        imdb_ratings.append(float(one_movie_info.strong.text))
        #메타스코어 np.nan
        if one_movie_info.find('span',class_='metascore') == None:
            meta_scores.append(np.nan)
        else:
            meta_scores.append(int(one_movie_info.find('span',class_='metascore').get_text().replace(' ','')))
        genres.append(one_movie_info.find('span',class_='genre').text.strip())
        movie_id.append(one_movie_info.h3.a['href'].split('/')[2])
            

        
        # 하나의 영화씩 선택 -> one_movie_info
        # one_movie_info.select_one('h3>a')
        # 연도
        # 시간
        # 평점
        # 메타스코어 np.nan
        # 5개의 원소가 들어가게

    # #movie_year
    # movie_year=soup.select('h3>span.lister-item-year')
    # for year in movie_year:
    #     years.append(year.get_text().strip())
    # #time
    # run_time=soup.select('p>span.runtime')
    # for runtime in run_time:
    #     runtimes.append(runtime.get_text().strip())
    # #strong(평점)
    # imdb_rating=soup.select('strong')[2:]
    # for rating in imdb_rating:
    #     imdb_ratings.append(float(rating.get_text().strip()))
    # #메타스코어(점수)
    
    # meta_score=soup.select('span.metascore')
    # for metascore in meta_score:
    #     meta=metascore.get_text().strip()
    #     if len(meta)>1:
    #         meta_scores.append(int(meta))
    #     else:
    #         meta_scores.append(np.nan)
    # #장르
    # genress=soup.select('span.genre')
    # for genre in genress:
    #     genres.append(genre.get_text().strip())



In [50]:
genres
len(movie_id)

250

In [51]:
meta_score=soup.select('span.metascore')
int(meta_score[0].get_text().strip())

70

In [52]:
df=pd.DataFrame([movie_id,names,years,runtimes,imdb_ratings,meta_scores,genres]).T
df.columns=['movie_id','name','year','runtime(min)','imdb_rating','meta_score','genre']
df['year']=df['year'].apply(lambda x : x.replace("(","").replace(")",""))
df['runtime(min)']=df['runtime(min)'].apply(lambda x: x[:3])
df['meta_score'].value_counts()

86     12
90     12
84      9
77      9
88      9
98      9
92      9
85      8
75      8
96      8
100     7
89      7
82      7
94      7
87      6
79      6
81      6
78      6
97      6
72      5
95      5
71      5
93      4
83      4
69      4
70      4
65      4
66      4
91      4
73      4
74      4
76      3
63      3
62      3
64      3
68      3
80      3
57      2
59      2
67      2
99      2
58      1
60      1
61      1
55      1
Name: meta_score, dtype: int64

In [53]:
df['meta_score'].isnull().sum()

18

In [56]:

# movie_year=soup.select('h3>span.lister-item-year')
# for year in movie_year:
#     years.append(year.get_text().strip())
# years

df
df.to_csv('movie_info2.csv',index=False)

In [ ]:
movie_name=soup.select('strong')[2:]
for rating in movie_name:
    rating.get_text().strip()


[<strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.

In [57]:
meta_score=soup.select('span.metascore')
for metascore in meta_score:
    meta=metascore.get_text().strip()

In [ ]:
genres=soup.select('span.genre')
for genre in genres:
    print(genre.get_text().strip())


Comedy, Crime, Drama
Biography, Crime, Drama
Drama, Thriller
Action, Comedy, Crime
Animation, Adventure, Comedy
Biography, Drama, Family
Crime, Drama, Mystery
Drama
Animation, Action, Adventure
Drama, War
Drama, War
Drama, Romance
Action, Drama, Sport
Action, Biography, Drama
Biography, Drama, History
Biography, Drama, Sport
Comedy, Drama, Thriller
Drama, Sport
Animation, Action, Adventure
Adventure, Drama, War
Biography, Drama, Family
Crime, Drama
Animation, Comedy, Family
Drama
Adventure, Drama
Drama, Romance
Crime, Drama, Thriller
Biography, Drama, History
Drama, Fantasy
Drama, Thriller
Biography, Crime, Drama
Adventure, Drama, War
Drama, Thriller
Drama, Film-Noir, Mystery
Comedy, Romance
Drama
Film-Noir, Mystery, Thriller
Animation, Comedy, Drama
Drama, Romance, War
Adventure, Comedy, Drama
Crime, Drama
Drama, Romance
Comedy, Drama
Drama, War
Action, Adventure, Comedy
Animation, Action, Adventure
Adventure, Drama, Western
Animation, Adventure, Comedy
Comedy, Drama, Fantasy
Comedy


In [ ]:
filtering=soup.select('h3>a')
for filter in filtering:
        print(filter['href'].split('/')[2])

tt5027774
tt1895587
tt3170832
tt0120735
tt0198781
tt0059742
tt0353969
tt1205489
tt0892769
tt0091763
tt0077416
tt0112471
tt1291584
tt1979320
tt2024544
tt0081398
tt3011894
tt0405159
tt0129167
tt0072684
tt1028532
tt0113247
tt0096283
tt0074958
tt0052618
tt0381681
tt0047296
tt0395169
tt0050976
tt0245712
tt0107207
tt0050212
tt0060827
tt0032976
tt0025316
tt0032551
tt0041959
tt0978762
tt0036868
tt0015864
tt0053198
tt0050986
tt0031679
tt0058946
tt0017925
tt0317705
tt0099348
tt0103639
tt0107048
tt0079470


In [ ]:
review_info=soup.select('#main div.review-container > div.lister-item-content')
listreview=[]
# > div.ipl-ratings-bar'
# rating_info=soup.select('div.lister-item-content > div.ipl-ratings-bar')
# review_info.select_one('div.ipl-ratings-bar').get_text().strip()
for reviewinfo in review_info:
    if reviewinfo.select('div.ipl-ratings-bar')==[]:
        print(np.nan)
    else:
        n=int(reviewinfo.select_one('div.ipl-ratings-bar').get_text().strip().split('/')[0])
        print(n)


# len(review_info)

# n=soup.select('.rating-other-user-rating>span')
# for i in range(0,len(n),2):
#     rating.append(int(n[i].get_text()))
# rating
# rating
# review_info.div
# review_info.find('span',class_='display-name-link').text
# review_info.select_one('span.display-name-link').get_text()

10
10
7
10
10
10
10
8
10
8
10
10
10
8
9
8
10
8
8
9
10
10
10
10
10


In [ ]:
len(review_info[:10])

10

In [73]:
id2=[]
user_id=[]
rating=[]
for id in movie_id:
    url="https://www.imdb.com/title/"+id+"/reviews/?ref_=tt_ql_2"
    driver.get(url)
    time.sleep(3)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    review_info=soup.select('#main div.review-container')
    rating_info=soup.select('div.lister-item-content > div.ipl-ratings-bar')
    for j in range(len(review_info)):
        id2.append(id)
        # user_ids=review_info.find('span',class_='display-name-link').text
        user_ids=review_info[j].select_one('span.display-name-link').get_text()
        user_id.append(user_ids)
        # rating
        review_info=soup.select('#main div.review-container > div.lister-item-content')
    for reviewinfo in review_info:
        if reviewinfo.select('div.ipl-ratings-bar')==[]:
            rating.append(np.nan)
        else:
            rating.append(int(reviewinfo.select_one('div.ipl-ratings-bar').get_text().strip().split('/')[0]))
    
        
    

6235

In [78]:
import pandas as pd
df2=pd.DataFrame([id2,user_id,rating]).T
df2.columns=['movie_id','user_id','rating']
df2['user_id'].value_counts()
# # df2.loc[df2['user_id']=='TheLittleSongbird'][['rating']]
df2.to_csv('user_info.csv',index=False)
# df2.head(10)
df2

,movie_id,user_id,rating
0,tt0111161,hitchcockthelegend,10
1,tt0111161,Sleepin_Dragon,10
2,tt0111161,EyeDunno,10
3,tt0111161,alexkolokotronis,10
4,tt0111161,kaspen12,10
...,...,...,...
6230,tt0079470,TheAll-SeeingI,10
6231,tt0079470,ptpenry,10
6232,tt0079470,riggo-73503,9
6233,tt0079470,waltersjon,10


In [105]:
combined=pd.merge(df2,df)
combined
pvt=combined.pivot_table(index='user_id',columns='name',values='rating').fillna(0)
pvt
item_corr=pvt.corr()
item_corr
combined.to_csv('movie_totalinfo.csv',index=False)
a=pd.DataFrame(df2['user_id'].value_counts()[:10]).reset_index()
a

,index,user_id
0,TheLittleSongbird,160
1,planktonrules,113
2,SnoopyStyle,84
3,Xstal,80
4,Hitchcoc,75
5,claudio_carvalho,70
6,Leofwine_draca,68
7,classicsoncall,66
8,bkoganbing,64
9,bob the moo,61


In [86]:
interested="Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb"
item_corr.sort_values(by=interested,ascending=False)[interested]


TypeError: DataFrame.sort_values() missing 1 required positional argument: 'by'

In [81]:
user_corr=pvt.T.corr()
user_corr

user_id,*DATo*,0w0,29055,3xHCCH,851222,AAdaSC,ACitizenCalledKane,ACollegeStudent,AJB2002,AKS-6,...,zeletto,zennokangae,zeph-3,zetes,zkonedog,zolaaar,zorba-36271,ztruk2001,zzhhhqing,zzyzx000-124-271558
user_id,,,,,,,,,,,,,,,,,,,,,
*DATo*,1.000000,-0.004016,-0.004016,-0.005656,-0.009926,-0.005397,-0.008047,-0.008967,-0.004016,-0.004016,...,-0.004016,-0.004016,-0.004016,-0.016870,-0.011444,-0.004016,-0.004016,-0.004016,-0.005683,-0.004016
0w0,-0.004016,1.000000,-0.004016,-0.005656,-0.009926,-0.005397,-0.008047,-0.008967,-0.004016,-0.004016,...,-0.004016,-0.004016,-0.004016,-0.016870,-0.011444,-0.004016,-0.004016,-0.004016,-0.005683,-0.004016
29055,-0.004016,-0.004016,1.000000,-0.005656,-0.009926,-0.005397,-0.008047,-0.008967,-0.004016,-0.004016,...,-0.004016,-0.004016,-0.004016,-0.016870,-0.011444,-0.004016,-0.004016,-0.004016,-0.005683,-0.004016
3xHCCH,-0.005656,-0.005656,-0.005656,1.000000,-0.013978,-0.007600,-0.011333,-0.012628,-0.005656,-0.005656,...,0.779889,-0.005656,-0.005656,-0.023759,-0.016117,-0.005656,-0.005656,-0.005656,-0.008004,-0.005656
851222,-0.009926,-0.009926,-0.009926,-0.013978,1.000000,-0.013338,-0.019888,-0.022161,-0.009926,-0.009926,...,-0.009926,-0.009926,-0.009926,0.042173,-0.028283,-0.009926,-0.009926,-0.009926,-0.014045,-0.009926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zolaaar,-0.004016,-0.004016,-0.004016,-0.005656,-0.009926,-0.005397,-0.008047,-0.008967,-0.004016,-0.004016,...,-0.004016,-0.004016,-0.004016,-0.016870,-0.011444,1.000000,-0.004016,-0.004016,-0.005683,-0.004016
zorba-36271,-0.004016,-0.004016,-0.004016,-0.005656,-0.009926,-0.005397,-0.008047,-0.008967,-0.004016,-0.004016,...,-0.004016,-0.004016,-0.004016,-0.016870,-0.011444,-0.004016,1.000000,-0.004016,-0.005683,-0.004016
ztruk2001,-0.004016,-0.004016,-0.004016,-0.005656,-0.009926,-0.005397,-0.008047,-0.008967,-0.004016,-0.004016,...,-0.004016,-0.004016,-0.004016,-0.016870,-0.011444,-0.004016,-0.004016,1.000000,-0.005683,-0.004016


In [82]:
target='TheLittleSongbird'
tmp=user_corr.sort_values(by=target,ascending=False)[target]
tmp


user_id
TheLittleSongbird    1.000000
MovieAddict2016      0.144428
Lechuguilla          0.134963
bkoganbing           0.134882
BA_Harrison          0.129992
                       ...   
FeastMode           -0.145899
Arcturus1980        -0.146075
BandSAboutMovies    -0.160968
deloudelouvain      -0.167548
kosmasp             -0.168128
Name: TheLittleSongbird, Length: 2509, dtype: float64

In [94]:
pd.Series([]).empty

C:\Users\Playdata\AppData\Local\Temp\ipykernel_19224\2352566002.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series([]).empty


True

In [83]:
a=tmp[tmp < 0.9999]
b=pd.DataFrame(a).reset_index()
list(b['user_id'].head(10))

df2.tail()

,movie_id,user_id,rating
6230,tt0079470,TheAll-SeeingI,10
6231,tt0079470,ptpenry,10
6232,tt0079470,riggo-73503,9
6233,tt0079470,waltersjon,10
6234,tt0079470,MrVibrating,10


In [85]:
target='TheLittleSongbird'
user_corr.sort_values(by=target,ascending=False)[target].head()
u1=set(combined.loc[combined['user_id']=='TheLittleSongbird']['name'])
u2=set(combined.loc[combined['user_id']=='MovieAddict2016']['name'])
u3=set(combined.loc[combined['user_id']=='boatsfra']['name'])
diff=u2.difference(u1)
u2_all=combined.loc[combined['user_id']=='Smells_Like_Cheese']
filtered=u2_all.loc[combined['name'].isin(diff)]
filtered.sort_values(by='rating',ascending=False).head()['name']




300     Il buono, il brutto, il cattivo
1053                 American History X
Name: name, dtype: object

In [ ]:
soup.select_one('div.lister-item-content').get_text()
n=list(soup.select('div.lister-item-content').strings)[8]
n

'10'

In [ ]:
# len(id2)
# len(user_id)
# len(rating)
soup.select('.rating-other-user-rating>span')

[<span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>9</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="

In [ ]:
#main > section > div.lister > div.lister-list > div:nth-child(34) > div.review-container > div.lister-item-content > div.ipl-ratings-bar > span > span:nth-child(2)

25

In [3]:
import pandas as pd

In [ ]:
import time
import numpy as np
names=[]
years=[]
runtimes=[]
imdb_ratings=[]
meta_scores=[]
genres=[]
movie_id=[]
pages=np.arange(1,251,50)
for page in pages:
    url=f'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start={page}&ref_=adv_nxt'
    time.sleep(3)
    driver.get(url)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    soup.prettify
    movies_info = soup.select('div.lister-item.mode-advanced')
    #이름
    for one_movie_info in movies_info:
        #제목
        names.append(one_movie_info.h3.a.get_text().strip())
        #연도
        years.append(one_movie_info.h3.find('span',class_='lister-item-year').text)
        #시간
        runtimes.append(one_movie_info.p.find('span',class_='runtime').text)
        #평점
        imdb_ratings.append(float(one_movie_info.strong.text))
        #메타스코어 np.nan
        if one_movie_info.find('span',class_='metascore') == None:
            meta_scores.append(np.nan)
        else:
            meta_scores.append(int(one_movie_info.find('span',class_='metascore').get_text().replace(' ','')))
        genres.append(one_movie_info.find('span',class_='genre').text.strip())
        movie_id.append(one_movie_info.h3.a['href'].split('/')[2])
            

        
        # 하나의 영화씩 선택 -> one_movie_info
        # one_movie_info.select_one('h3>a')
        # 연도
        # 시간
        # 평점
        # 메타스코어 np.nan

In [ ]:
id2=[]
user_id=[]
rating=[]
for id in movie_id:
    url="https://www.imdb.com/title/"+id+"/reviews/?ref_=tt_ql_2"
    driver.get(url)
    time.sleep(3)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    review_info=soup.select('#main div.review-container')
    rating_info=soup.select('div.lister-item-content > div.ipl-ratings-bar')
    for j in range(len(review_info)):
        #id2
        id2.append(id)
        # user_ids=review_info.find('span',class_='display-name-link').text
        user_ids=review_info[j].select_one('span.display-name-link').get_text()
        user_id.append(user_ids)
        # rating
        review_info=soup.select('#main div.review-container > div.lister-item-content')
    for reviewinfo in review_info:
        if reviewinfo.select('div.ipl-ratings-bar')==[]:
            rating.append(np.nan)
        else:
            rating.append(int(reviewinfo.select_one('div.ipl-ratings-bar').get_text().strip().split('/')[0]))
    
        

In [6]:
df=pd.read_csv('movie_info2.csv')
df

,movie_id,name,year,runtime(min),imdb_rating,meta_score,genre
0,tt0111161,The Shawshank Redemption,1994,142,9.3,82.0,Drama
1,tt0068646,The Godfather,1972,175,9.2,100.0,"Crime, Drama"
2,tt0468569,The Dark Knight,2008,152,9.0,84.0,"Action, Crime, Drama"
3,tt0108052,Schindler's List,1993,195,9.0,95.0,"Biography, Drama, History"
4,tt0071562,The Godfather Part II,1974,202,9.0,90.0,"Crime, Drama"
...,...,...,...,...,...,...,...
245,tt0317705,The Incredibles,2004,115,8.0,90.0,"Animation, Action, Adventure"
246,tt0103639,Aladdin,1992,90,8.0,86.0,"Animation, Adventure, Comedy"
247,tt0099348,Dances with Wolves,1990,181,8.0,72.0,"Adventure, Drama, Western"
248,tt0107048,Groundhog Day,1993,101,8.0,72.0,"Comedy, Drama, Fantasy"


In [8]:
df2=pd.read_csv('user_info.csv')
df2

,movie_id,user_id,rating
0,tt0111161,hitchcockthelegend,10.0
1,tt0111161,Sleepin_Dragon,10.0
2,tt0111161,EyeDunno,10.0
3,tt0111161,alexkolokotronis,10.0
4,tt0111161,kaspen12,10.0
...,...,...,...
6230,tt0079470,TheAll-SeeingI,10.0
6231,tt0079470,ptpenry,10.0
6232,tt0079470,riggo-73503,9.0
6233,tt0079470,waltersjon,10.0


In [10]:
a=pd.DataFrame(df2['user_id'].value_counts()[:10]).reset_index()
a

,index,user_id
0,TheLittleSongbird,160
1,planktonrules,113
2,SnoopyStyle,84
3,Xstal,80
4,Hitchcoc,75
5,claudio_carvalho,70
6,Leofwine_draca,68
7,classicsoncall,66
8,bkoganbing,64
9,bob the moo,61


In [12]:
combined=pd.merge(df2,df)
combined
# pvt=combined.pivot_table(index='user_id',columns='name',values='rating').fillna(0)
# pvt

,movie_id,user_id,rating,name,year,runtime(min),imdb_rating,meta_score,genre
0,tt0111161,hitchcockthelegend,10.0,The Shawshank Redemption,1994,142,9.3,82.0,Drama
1,tt0111161,Sleepin_Dragon,10.0,The Shawshank Redemption,1994,142,9.3,82.0,Drama
2,tt0111161,EyeDunno,10.0,The Shawshank Redemption,1994,142,9.3,82.0,Drama
3,tt0111161,alexkolokotronis,10.0,The Shawshank Redemption,1994,142,9.3,82.0,Drama
4,tt0111161,kaspen12,10.0,The Shawshank Redemption,1994,142,9.3,82.0,Drama
...,...,...,...,...,...,...,...,...,...
6230,tt0079470,TheAll-SeeingI,10.0,Life of Brian,1979,94,8.0,77.0,Comedy
6231,tt0079470,ptpenry,10.0,Life of Brian,1979,94,8.0,77.0,Comedy
6232,tt0079470,riggo-73503,9.0,Life of Brian,1979,94,8.0,77.0,Comedy
6233,tt0079470,waltersjon,10.0,Life of Brian,1979,94,8.0,77.0,Comedy
